In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from modules.data_module import KsatDataModule
from pprint import pprint
from omegaconf import OmegaConf

pd.set_option('display.max_columns', None)

### 모델, 토크나이저 불러오기

템플릿 설정과 스페셜 토큰 설정을 모델 별로 자동화하기 곤란한 부분이 있어 직접 확인 후 `model.py`에 적절한 설정을 추가해주기 위해 필요한 절차에 대한 설명입니다.

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "Bllossom/llama-3.2-Korean-Bllossom-3B",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(
    "Bllossom/llama-3.2-Korean-Bllossom-3B",
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### chat template 확인

In [7]:
pprint(tokenizer.chat_template)

('{{- bos_token }}\n'
 '{%- if custom_tools is defined %}\n'
 '    {%- set tools = custom_tools %}\n'
 '{%- endif %}\n'
 '{%- if not tools_in_user_message is defined %}\n'
 '    {%- set tools_in_user_message = true %}\n'
 '{%- endif %}\n'
 '{%- if not date_string is defined %}\n'
 '    {%- if strftime_now is defined %}\n'
 '        {%- set date_string = strftime_now("%d %b %Y") %}\n'
 '    {%- else %}\n'
 '        {%- set date_string = "26 Jul 2024" %}\n'
 '    {%- endif %}\n'
 '{%- endif %}\n'
 '{%- if not tools is defined %}\n'
 '    {%- set tools = none %}\n'
 '{%- endif %}\n'
 '\n'
 '{#- This block extracts the system message, so we can slot it into the right '
 'place. #}\n'
 "{%- if messages[0]['role'] == 'system' %}\n"
 "    {%- set system_message = messages[0]['content']|trim %}\n"
 '    {%- set messages = messages[1:] %}\n'
 '{%- else %}\n'
 '    {%- set system_message = "You are a helpful AI assistant. Please answer '
 "the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 

chat_template이 None일 경우, 베이스라인 모델처럼 chat_template을 설정해주세요.

In [8]:
tokenizer.special_tokens_map

{'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}

In [3]:
config = OmegaConf.load("/data/ephemeral/home/JH/level2-nlp-generationfornlp-nlp-04-lv3/config/config.yaml")
data_module = KsatDataModule(tokenizer, config.data)
data_module.setup("train")

Converting to prompts: 100%|██████████| 1634/1634 [00:00<00:00, 9400.54it/s]


Tokenizing (num_proc=4):   0%|          | 0/1634 [00:00<?, ? examples/s]

Converting to prompts: 100%|██████████| 397/397 [00:00<00:00, 8917.12it/s]


Tokenizing (num_proc=4):   0%|          | 0/397 [00:00<?, ? examples/s]

In [4]:
print(tokenizer.decode(data_module.train_examples[0]["input_ids"], skip_special_tokens=False))

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 15 Nov 2024

지문을 읽고 질문의 답을 구하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

지문:
            상소하여 아뢰기를, “신이 좌참 찬 송준길이 올린 차자를 보았는데, 상복(喪服) 절차에 대하여 논한 것이 신과는 큰 차이가 있었습니다. 장자를 위하여 3년을 입는 까닭은 위로 ‘정체(正體)’가 되기 때문이고 또 전 중(傳重: 조상의 제사나 가문의 법통을 전함)하기 때문입니다. …(중략) … 무엇보다 중요한 것은 할아버지와 아버지의 뒤를 이은 ‘정체’이지, 꼭 첫째이기 때문에 참 최 3년 복을 입는 것은 아닙니다.”라고 하였다.－현종실록 －ㄱ.기 사환국으로 정권을 장악하였다.ㄴ.인 조반정을 주도 하여 집권세력이 되었다.ㄷ.정조 시기에 탕평 정치의 한 축을 이루었다.ㄹ.이 이와 성혼의 문인을 중심으로 형성되었다.

            질문:
            상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?

            선택지:
            1 - ㄱ, ㄷ
2 - ㄱ, ㄴ
3 - ㄴ, ㄹ
4 - ㄷ, ㄹ

            1, 2, 3, 4, 5 중에 하나를 정답으로 고르세요.
            정답:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

1<|eot_id|>


여기서 "정답 번호\<eos token>" 위의 일부분을 복사해서 `model.py`의 `_get_response_template` 메서드에 response_template으로 추가해주세요.

예를 들어,
```
...

1, 2, 3, 4, 5 중에 하나를 정답으로 고르세요.
정답:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

1<|eot_id|>
```


템플릿이 이렇게 되어 있으면 "1<|eot_id|>" 위의 

"<|start_header_id|>assistant<|end_header_id|>\n\n" 정도를 복사해서 response_template으로 설정하면 됩니다.

In [13]:
for token_id in data_module.train_examples[0]["input_ids"][-10:]:
    print(tokenizer.decode([token_id], skip_special_tokens=False))

 정
답
:
<|eot_id|>
<|start_header_id|>
assistant
<|end_header_id|>



1
<|eot_id|>


In [11]:
tokenizer.special_tokens_map

{'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}

모델의 토크나이저에 설정된 스페셜 토큰 목록과 템플릿 디코딩 결과를 보고, 정답 번호 뒤 마지막 토큰이 한 단위로 안 묶여 있으면 `model.py`의 `_get_special_tokens` 메서드에서 스페셜 토큰으로 추가해주시면 됩니다.